In [1]:
import sys
import pickle
sys.path.append('../')
from models.LSTM.lstm import F1LapTimePredictor

In [2]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score, mean_squared_error
import pickle
import os
from datetime import datetime

def save_training_artifacts(model, study, save_dir='models'):
    """Save model, hyperparameters, and study results."""
    # Create directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)
    
    # Generate timestamp for unique filenames
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Save model weights and architecture
    model_path = os.path.join(save_dir, f'lstm_model_{timestamp}.pt')
    torch.save({
        'model_state_dict': model.state_dict(),
        'static_feature_size': model.static_network[0].in_features,  # Already an integer
        'dynamic_feature_size': model.lstm.input_size,
        'hidden_size': model.lstm.hidden_size,
        'num_layers': model.lstm.num_layers,
        'dropout': model.lstm.dropout if isinstance(model.lstm.dropout, float) else 0.2
    }, model_path)
    
    # Save study results and hyperparameters
    study_results = {
        'best_params': study.best_params,
        'best_value': study.best_value,
        'n_trials': len(study.trials),
        'study_statistics': {
            'best_trial': study.best_trial.number,
            'datetime': timestamp,
            'trials_dataframe': study.trials_dataframe().to_dict() if hasattr(study, 'trials_dataframe') else None
        }
    }
    
    study_path = os.path.join(save_dir, f'lstm_study_{timestamp}.pkl')
    with open(study_path, 'wb') as f:
        pickle.dump(study_results, f)
    
    print(f"Model saved to: {model_path}")
    print(f"Study results saved to: {study_path}")
    
    return model_path, study_path

def evaluate_and_visualize(predictor, processed_data, save_dir='models'):
    """Evaluate model performance and create visualization."""
    # Create val_loader using predictor's current parameters
    val_loader = predictor.create_data_loaders(
        batch_size=predictor.best_params['batch_size'], 
        sequence_length=predictor.best_params['sequence_length']
    )[1]
    
    actual_times = []
    predicted_times = []
    race_ids = []
    driver_ids = []
    lap_numbers = []
    
    # Use predictor.best_model instead of predictor.model
    predictor.best_model.eval()
    with torch.no_grad():
        for batch in val_loader:
            dynamic_seq = batch['dynamic_sequence'].to(predictor.device)
            static_feat = batch['static_features'].to(predictor.device)
            targets = batch['target']
            
            predictions = predictor.best_model(dynamic_seq, static_feat)
            
            actual_times.extend(targets.numpy())
            predicted_times.extend(predictions.cpu().numpy())
            
            # Get metadata for current batch
            metadata_indices = batch.get('metadata_indices', [])
            if metadata_indices:
                test_metadata = processed_data['test']['metadata']
                race_ids.extend(test_metadata.iloc[metadata_indices]['raceId'].values)
                driver_ids.extend(test_metadata.iloc[metadata_indices]['driverId'].values)
                lap_numbers.extend(test_metadata.iloc[metadata_indices]['lap'].values)
    
    # Rest of the function remains the same...

# Main execution
with open('processed_race_data.pkl', 'rb') as f:
    processed_data = pickle.load(f)

# Initialize the predictor
predictor = F1LapTimePredictor(processed_data)

# Run optimization
study = predictor.optimize(n_trials=2)

# Create and store the best model in the predictor
predictor.best_model = predictor.create_best_model(study)
predictor.best_params = study.best_params  # Store best parameters

# Save training artifacts
model_path, study_path = save_training_artifacts(predictor.best_model, study)

# evaluation
evaluate_and_visualize(predictor, processed_data)

[I 2024-12-08 18:34:00,345] A new study created in memory with name: no-name-509b15e3-2f4f-4899-8deb-6a036de0f2a6


Using device: mps
Model is on correct device: mps:0


/Users/I551659/Library/Caches/pypoetry/virtualenvs/ie500-data-mining-group7-LKR-OXJO-py3.12/lib/python3.12/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/Users/I551659/Library/Caches/pypoetry/virtualenvs/ie500-data-mining-group7-LKR-OXJO-py3.12/lib/python3.12/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker

Using device: mps
Model is on correct device: mps:0


/Users/I551659/Library/Caches/pypoetry/virtualenvs/ie500-data-mining-group7-LKR-OXJO-py3.12/lib/python3.12/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
[I 2024-12-08 19:09:11,881] Trial 1 finished with value: 0.5292047262191772 and parameters: {'hidden_size': 134, 'num_layers': 3, 'dropout': 0.2635489357835732, 'sequence_length': 8, 'batch_size': 62, 'learning_rate': 0.0008187608111383791, 'weight_decay': 5.044775595466323e-05}. Best is trial 1 with value: 0.5292047262191772.


Model saved to: models/lstm_model_20241208_190911.pt
Study results saved to: models/lstm_study_20241208_190911.pkl


/Users/I551659/Library/Caches/pypoetry/virtualenvs/ie500-data-mining-group7-LKR-OXJO-py3.12/lib/python3.12/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


AttributeError: 'F1LapTimePredictor' object has no attribute 'model'